In diesem Szenario wird in Town05 (Spawn_point(2)) randomisierte statische Objekte auf der Straße plaziert.

In [ ]:
%pip install carla

In [ ]:
import carla
import math
import random
import time
import numpy as np
import cv2
import open3d as o3d 
from matplotlib import cm 

In [ ]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

#Load a World
client.load_world('Town05')


In [ ]:
#Add the ego_vehicle
vehicle_bp_benz= bp_lib.find('vehicle.mercedes.coupe_2020')
vehicle_bp_benz.set_attribute('color', '255,255,255')
ego_vehicle = world.spawn_actor(vehicle_bp_benz, spawn_points[31])

In [ ]:
#Move spectator behind to view
spectator = world.get_spectator()
transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=-6, z=2.0)), ego_vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [ ]:
#Iterate this cell to find desired camera location
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2)) #Change this to move camera
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

#time.sleep(0.2)
#camera.destroy()

In [ ]:
def spawn_anomaly_ahead(self, distance):
    transform = carla.Transform(self.get_transform().transform(carla.Location(x = distance)))
    spawn_anomaly(transform)

def spawn_anomaly_freePosition(self, distance_x, distance_y, distance_z):
    transform = carla.Transform(self.get_transform().transform(carla.Location(x = distance_x, y = distance_y, z = distance_z)))
    spawn_anomaly(transform)

def spawn_anomaly(transform):
    bp_static = world.get_blueprint_library().filter('static.prop.*')
    anomaly_object = random.choice(bp_static)
    player = world.try_spawn_actor(anomaly_object,transform)
    



In [ ]:
spawn_anomaly_freePosition(ego_vehicle, 40, 0, 0)

In [ ]:
spawn_anomaly_freePosition(ego_vehicle, 20, -5, 0)

In [ ]:
spawn_anomaly_freePosition(ego_vehicle, 30, 2, 0)

In [ ]:
# Spawn camera
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)
cameraDisk = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)
# Callback stores sensor data in a dictionary for use outside callback                         
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Get gamera dimensions and initialise dictionary                       
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'image': np.zeros((image_h, image_w, 4))}

# Start camera recording
camera.listen(lambda image: camera_callback(image, camera_data))
cameraDisk.listen(lambda image: image.save_to_disk('outONE/%06d.png' % image.frame))

In [ ]:
# Set traffic in motion
for vehicle in world.get_actors().filter('*vehicle*'): 
    vehicle.set_autopilot(True) 

In [ ]:
# OpenCV named window for rendering
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

# Close OpenCV window when finished
camera.stop()
cv2.destroyAllWindows()
cv2.stop()